In [3]:
%%time
# core importations
from annotationframeworkclient import FrameworkClient
import pandas as pd

# dash importations
import dash
from dash import Dash, dcc, html, Input, Output, State
import plotly.graph_objects as go
import plotly.express as px


# ----------CORE DATA GATHERING FUCNTIONS----------

# defines function to convert nm coordinates into FlyWire-usable
def coordConvert(coords):
    x = coords
    x[0] /= 4
    x[1] /= 4
    x[2] /= 40
    return x

# defines function to get nucleus table entry using root id
def getNuc(root):
    client = FrameworkClient("flywire_fafb_production")
    mat_vers = max(client.materialize.get_versions())
    nuc_df = client.materialize.query_table('nuclei_v1',
                                            filter_in_dict={"pt_root_id": [root]},
                                            materialization_version = mat_vers)
    return nuc_df 

# defines function to get synapse table entry using root id
def getSyn(root):
    client = FrameworkClient("flywire_fafb_production")
    mat_vers = max(client.materialize.get_versions())
    syn_df = client.materialize.query_table('synapses_nt_v1', 
                                            filter_in_dict={"pre_pt_root_id":[root]})
    return len(syn_df)

# defines function to get edit data
def getEdits(root):
    client = FrameworkClient("flywire_fafb_production")
    changes_dict = client.chunkedgraph.get_change_log(int(root))
    return changes_dict   

# defines function to build dataframe using root id list
def dfBuilder(root_list):
    data_list = []
    
    #TEMP FOR TESTING#
    print(root_list)
    print(type(root_list[0]))
    
    for i in root_list:
        sub_list = []
        sub_list.append(i) #adds root#
        nuc_df = getNuc(i) #pulls nucleus table#
        change_dict = getEdits(i) #pulls changelog#
        sub_list.append(nuc_df.loc[0,'id']) #adds nucleus id #
        sub_list.append(coordConvert(nuc_df.loc[0,'pt_position'])) #adds coordinates#
        sub_list.append(nuc_df.loc[0,'volume']) #adds volume#
        sub_list.append(getSyn(i)) #adds synapses#
        sub_list.append(change_dict['n_splits']) #adds splits# 
        sub_list.append(change_dict['n_mergers']) #adds mergers#
        sub_list.append((sub_list[-1] + sub_list[-2])) #adds edits#
        sub_list.append(change_dict['user_info']) #adds editors#
        data_list.append(sub_list) #adds entry to data_list#

    # converts data_list into dataframe
    df = pd.DataFrame(data_list,columns = ['root_id','nucleus_id','nucleus_coordinates','volume','synapses',
                                           'mergers','splits','edits','editors'])
    return df

# ----------CORE DATA ASSIGNMENTS----------

# assigns root id
roots = [720575940628719355,720575940618059861,720575940618617536]
df = dfBuilder(roots)
df.head()



    

    

[720575940628719355, 720575940618059861, 720575940618617536]
<class 'int'>


<timed exec>:56: FutureWarning: 'pyarrow.deserialize' is deprecated as of 2.0.0 and will be removed in a future version. Use pickle or the pyarrow IPC functionality instead.
<timed exec>:61: FutureWarning: 'pyarrow.deserialize' is deprecated as of 2.0.0 and will be removed in a future version. Use pickle or the pyarrow IPC functionality instead.


Wall time: 5.21 s


,root_id,nucleus_id,nucleus_coordinates,volume,synapses,mergers,splits,edits,editors
0,720575940628719355,3735774,"[117816, 70936, 1194]",23.723213,1432,1,24,25,"{'13': {'n_mergers': 2, 'n_splits': 1}, '96': ..."
1,720575940618059861,3735989,"[118664, 70864, 1244]",25.629778,61,0,1,1,"{'1211': {'n_mergers': 1, 'n_splits': 0}}"
2,720575940618617536,3735879,"[118240, 69928, 1197]",26.161234,1375,4,2,6,"{'14': {'n_mergers': 2, 'n_splits': 2}, '17': ..."


In [5]:
# -----DASH BUTTON-----
from dash import Dash, dcc, html, Input, Output, State

app = Dash(__name__)

app.layout = html.Div([
    html.Div(dcc.Input(id='input-on-submit', type='number')), #formatting for input box#
    html.Button('Submit', id='submit-val', n_clicks=0), #formatting for button#
    html.Div(id='container-button-basic',
             children='Enter a value and press submit') #formatting for output text CHANGE TO DF#
])


@app.callback(
    Output('container-button-basic', 'children'), #defines output location as the 'children' aspect of 'container-button-basic'#
    Input('submit-val', 'n_clicks'), #defines trigger as button press (change in the state of the 'n_clicks' aspect of submit-val')# 
    State('input-on-submit', 'value') #defines state as the value of 'input-on-submit'#
)
#defines function that executes when value of 'n_clicks' changes#
#takes value of 'State' as argument, returns output to 'Output' location above#
def update_output(n_clicks, root):
    return dfBuilder(list(root))
# NEED TO FIND A WAY TO CONVERT INPUT STRING TO LIST OF INTS#


if __name__ == '__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [18/Oct/2021 17:07:41] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [18/Oct/2021 17:07:42] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [18/Oct/2021 17:07:42] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [18/Oct/2021 17:07:42] "GET /_favicon.ico?v=2.0.0 HTTP/1.1" 200 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\Moose\anaconda3\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\Moose\anaconda3\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\Moose\anaconda3\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\Moose\anaconda3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\Users\Moose\anaconda3\lib\site-packages\flask\app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\Moose\anaconda3\lib\site-packages\flask\app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "C:\Users\Moose\anaconda3\lib\site-packages\dash\dash.py", line 1336, in dispatch
    response.set_d

127.0.0.1 - - [18/Oct/2021 17:07:42] "POST /_dash-update-component HTTP/1.1" 500 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\Moose\anaconda3\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\Moose\anaconda3\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\Moose\anaconda3\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\Moose\anaconda3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\Users\Moose\anaconda3\lib\site-packages\flask\app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\Moose\anaconda3\lib\site-packages\flask\app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "C:\Users\Moose\anaconda3\lib\site-packages\dash\dash.py", line 1336, in dispatch
    response.set_d

127.0.0.1 - - [18/Oct/2021 17:07:44] "POST /_dash-update-component HTTP/1.1" 500 -


In [ ]:
# ----------DASH----------
def generate_table(dataframe, max_rows=10):
    return html.Table([
        html.Thead(
            html.Tr([html.Th(col) for col in dataframe.columns])
        ),
        html.Tbody([
            html.Tr([
                html.Td(dataframe.iloc[i][col]) for col in dataframe.columns
            ]) for i in range(min(len(dataframe), max_rows))
        ])
    ])

app = dash.Dash()

app.layout = html.Div([
    html.H4(children='US Agriculture Exports (2011)'),
    generate_table(df)
])

if __name__ == '__main__':
    app.run_server()